First, connect to our database by running the cell below (put your own username as the database user):

In [2]:
import datajoint as dj
dj.config['database.host'] = "gl-ash.biostr.washington.edu"
dj.config['database.user'] = "anna"
dj.config['database.port'] = 3306
dj.conn()

[2025-02-04 10:19:30,908][INFO]: Connecting anna@gl-ash.biostr.washington.edu:3306
[2025-02-04 10:19:30,935][INFO]: Connected anna@gl-ash.biostr.washington.edu:3306


DataJoint connection (connected) anna@gl-ash.biostr.washington.edu:3306

Next, import some useful spyglass packages and supress some warnings.

In [3]:
# ignore datajoint+jupyter async warnings
import warnings
warnings.simplefilter("ignore", category=DeprecationWarning)
warnings.simplefilter("ignore", category=ResourceWarning)
warnings.simplefilter("ignore", category=UserWarning)

# spyglass.common has the most frequently used tables
import spyglass.common as sgc

# spyglass.data_import has tools for inserting NWB files into the database
import spyglass.data_import as sgi

[10:19:37][WARNING] Spyglass: Please update position_tools to >= 0.1.0


Information and data in our database is stored in TABLES. For instance, `sgc.LabMember` is a table containing the list of lab members. To view the contents of a table, call it with parentheses at the end. Some tables have associated [part tables](https://datajoint.com/docs/core/datajoint-python/0.14/design/tables/master-part/) that contain more data. Not every entry in a table must have an entry in the part table, but every entry in the part table MUST have an entry in the master table. The [primary key](https://datajoint.com/docs/core/datajoint-python/0.14/design/tables/primary/) for an entry in a table is the set of attributes (each table column is an attribute) that unambiguously specify a given entry. These are designated by the bold column header.  

In [10]:
sgc.LabTeam()

team_name,team_description
Alex Garcia,
Anna Gillespie,
Gabby Shvartsman,test
Ian O'Leary,
Jason Tran,test
Josiah Quinn,test
Molly Park,test
My Team,test


Add yourself to our list of lab members.  Importantly, use a gmail address here. 

In [ ]:
sgc.LabMember.LabMemberInfo.insert(
    [  # Full name, Google email address, DataJoint username
        ["Firstname Lastname", "example1@gmail.com", "example1"]
    ],
    skip_duplicates=True,
)
sgc.LabMember.LabMemberInfo()

In [7]:
sgc.LabMember.LabMemberInfo()

lab_member_name,google_user_name For permission to curate,datajoint_user_name For permission to delete ns
Alex Garcia,jesusagarcia2410@gmail.com,alex
Anna Gillespie,akgillespie8@gmail.com,anna
Gabby Shvartsman,gabbyshv@gmail.com,gabby
Jason Tran,jasontranasaur@gmail.com,jason
Josiah Quinn,josiahquinn13@gmail.com,josiah
Molly Park,mollysypark@gmail.com,molly
